In [2]:
import tensorflow as tf

# Load model dari file .h5
model = tf.keras.models.load_model("model/mymodele.h5")

# Konversi ke format TFLite dengan opsi tambahan
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,       # Ops bawaan TFLite
    tf.lite.OpsSet.SELECT_TF_OPS          # Ops tambahan dari TensorFlow (agar LSTM dan TensorList bisa diproses)
]
converter._experimental_lower_tensor_list_ops = False  # Hindari konversi TensorList secara paksa

# Konversi
tflite_model = converter.convert()

# Simpan hasil .tflite
with open("mymodele.tflite", "wb") as f:
    f.write(tflite_model)

print("Konversi selesai! File disimpan sebagai mymodele.tflite")


INFO:tensorflow:Assets written to: C:\Users\gabry\AppData\Local\Temp\tmp8ulcyuot\assets


INFO:tensorflow:Assets written to: C:\Users\gabry\AppData\Local\Temp\tmp8ulcyuot\assets


Konversi selesai! File disimpan sebagai mymodele.tflite


In [1]:
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path='mymodele.tflite')
interpreter.allocate_tensors()
tensor_details = interpreter.get_tensor_details()
for tensor in tensor_details:
    print(tensor)

{'name': 'serving_default_masking_input:0', 'index': 0, 'shape': array([  1, 930,  13]), 'shape_signature': array([ -1, 930,  13]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
{'name': 'sequential/masking/Squeeze', 'index': 1, 'shape': array([2]), 'shape_signature': array([2]), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
{'name': 'strided_slice_1', 'index': 2, 'shape': array([3]), 'shape_signature': array([3]), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
{'name': 'strided_slice_11

In [2]:
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path='mymodele.tflite')
interpreter.allocate_tensors()
for op in interpreter._get_ops_details():
    print(op)

{'index': 0, 'op_name': 'NOT_EQUAL', 'inputs': array([ 0, 23]), 'outputs': array([36])}
{'index': 1, 'op_name': 'REDUCE_ANY', 'inputs': array([36, 27]), 'outputs': array([37])}
{'index': 2, 'op_name': 'CAST', 'inputs': array([37]), 'outputs': array([38])}
{'index': 3, 'op_name': 'MUL', 'inputs': array([ 0, 38]), 'outputs': array([39])}
{'index': 4, 'op_name': 'SHAPE', 'inputs': array([39]), 'outputs': array([40])}
{'index': 5, 'op_name': 'STRIDED_SLICE', 'inputs': array([40, 21, 22, 22]), 'outputs': array([41])}
{'index': 6, 'op_name': 'PACK', 'inputs': array([41, 24]), 'outputs': array([42])}
{'index': 7, 'op_name': 'FILL', 'inputs': array([42, 23]), 'outputs': array([43])}
{'index': 8, 'op_name': 'RESHAPE', 'inputs': array([37,  1]), 'outputs': array([44])}
{'index': 9, 'op_name': 'EXPAND_DIMS', 'inputs': array([44, 27]), 'outputs': array([45])}
{'index': 10, 'op_name': 'FULLY_CONNECTED', 'inputs': array([43, 32, -1]), 'outputs': array([46])}
{'index': 11, 'op_name': 'TRANSPOSE', 'in

In [3]:
import tensorflow as tf
import numpy as np

interpreter = tf.lite.Interpreter(model_path='mymodele.tflite')
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare dummy input (matching [1, 930, 13])
dummy_input = np.zeros((1, 930, 13), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], dummy_input)

# Run inference
interpreter.invoke()

# Get output
output = interpreter.get_tensor(output_details[0]['index'])
print("Output:", output)

Output: [[0.1266597  0.1602299  0.16510385 0.1393406  0.11905956 0.13949294
  0.1501134 ]]


In [4]:
import tensorflow as tf
import numpy as np

interpreter = tf.lite.Interpreter(model_path='mymodele.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

dummy_input = np.zeros((1, 930, 13), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], dummy_input)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])
print("Dummy output:", output)


Dummy output: [[0.1266597  0.1602299  0.16510385 0.1393406  0.11905956 0.13949294
  0.1501134 ]]
